In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pandas_ta
!pip install -U scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 103.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 118.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 15.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.43.0
    Uninstalling llvmlite-0.43.0:
      Successfully uninstalled llvmlite-0.43.0
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully unins

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 79.1 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
^C


In [29]:
!pip install permetrics==1.3.0

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error, r2_score
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM , GRU
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
import math
import seaborn as sns
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from pandas.core.frame import DataFrame
import tensorflow as tf
import keras
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.optimizers import Adam
from keras.layers import Conv1D, Conv2D, MaxPooling2D, MaxPooling1D
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.preprocessing import MinMaxScaler
import csv
import collections
from scipy.stats import zscore
from datetime import datetime
import tensorflow as tf
import collections
from scipy.stats import zscore
from datetime import datetime
from permetrics.regression import RegressionMetric
import pickle
from keras.layers import Embedding, Bidirectional, SimpleRNN

In [3]:
print(keras.__version__)
print(tf.__version__)

3.10.0
2.19.0


## Required functions

### Company data feature engineering functions

In [17]:
# fill missing values with a value at the same time one day ago
def fill_missing2(values):
 one_day = 1
 for row in range(values.shape[0]):
  for col in range(values.shape[1]):
    if np.isnan(values[row, col]):
     values[row, col] = values[row - one_day, col]

def fill_missing(values):
  return pd.DataFrame(values).fillna(method='ffill').values

def normalized_df(df):
    normalized_df=(df-df.mean())/df.std()
    return normalized_df

#def zlema(src, length):
#    lag = round((length - 1) / 2)
#    ema_data = src + (src - src[lag])
#    zl= ta.ema(ema_data,length = 5)
#    return zl


In [18]:
def get_company_df(dataset,company_name,start_date):
  company = dataset[(dataset['trading_name '] == company_name)]
  new_date_range = pd.date_range(start=start_date, end="2020-03-05", freq="D")
  company = company.reindex(new_date_range, fill_value=np.nan)
  company = company.drop(["symbol",'name','trading_name ','sectoer'], axis='columns')
  company.replace('?', np.nan, inplace=True)
  company = company.astype('float32')
  company[:] = fill_missing(company.values)
  company['zg'],company['sg'],company['xg'],company['atr'] = ta.aberration(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['lower'],company['mid'],company['upper'] = ta.accbands(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['ad'] = ta.ad(high=company['high'],low= company['low'],close=company['close'],volume=company['volume_traded '],length=5)
  company['adosc'] = ta.adosc(high=company['high'],low= company['low'],volume=company['volume_traded '],close=company['close'],fast=1,slow=5)
  # company['adx'],company['dmp'],company['dmn'] = ta.adx(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['alma'] = ta.alma(company['close'],length=5)
  company['amat1'], company['amat2']= ta.amat(company['close'],length=5)
  # company['ao'] = ta.ao(high=company['high'],low= company['low'],close=company['close'],fast=1,slow=5)
  company['aobv1'],company['aobv2'],company['aobv3'],company['aobv4'],company['aobv5'],company['aobv6'],company['aobv7'] = ta.aobv(company['close'],company['volume_traded '])
  # company['apo'] = ta.apo(company['close'],fast=1,slow=5)
  company['aroon_up'],company['aroon_down'],company['aroon_osc'] = ta.aroon(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['atr']= ta.atr(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['bbands_lower'],company['bbands_mid'],company['bbands_upper'],company['bbands_bandwidth'],company['bbands_percent'] = ta.bbands(company['close'],length=5)
  company['bias'] = ta.bias(company['close'],length=5)
  company['bop']= ta.bop(open_=company['open'],high=company['high'],low= company['low'],close=company['close'],length=5)
  company['ar'],company['br']= ta.brar(open_ = company['open'],high=company['high'],low= company['low'],close=company['close'],length=5)
  company['cci']= ta.cci(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['cfo'] = ta.cfo(company['close'],length=5)
  company['cg'] = ta.cg(company['close'],length=5)
  company['chop']= ta.chop(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['cksp_long'], company['cksp_short']= ta.cksp(high=company['high'],low= company['low'],close=company['close'])
  company['cmf']= ta.cmf(high=company['high'],low= company['low'],close=company['close'],volume = company['volume_traded '],open = company['open'],length=5)
  company['cmo'] = ta.cmo(company['close'],length=5)
  company['coppock'] = ta.coppock(company['close'],length=5,fast=6,slow=11)
  company['cti'] = ta.cti(company['close'],length=5)
  company['decay'] = ta.decay(company['close'],length=5)
  company['decreasing'] = ta.decreasing(company['close'],length=5)
  company['dema'] = ta.dema(company['close'],length=5)
  company['dmp'],company['dmn']= ta.dm(high=company['high'],low= company['low'],length=5)
  company['doncian_lower'],company['doncian_mid'],company['doncian_upper']= ta.donchian(high=company['high'],low= company['low'],lower_length=5,upper_length=5)
  company['dpo'] = ta.dpo(company['close'],length=5)
  company['ebsw'] = ta.ebsw(company['close'],length=5)
  company['efi'] = ta.efi(company['close'],company['volume_traded '],length=5)
  company['ema'] = ta.ema(company['close'],length=5)
  company['entropy'] = ta.entropy(company['close'],length=5)
  company['eom']= ta.eom(high=company['high'],low= company['low'],close=company['close'],volume = company['volume_traded '],length=5)
  company['er'] = ta.er(company['close'],length=5)
  company['eri_bull'],company['eri_bear'] = ta.eri(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['fisher1'],company['fisher2'] = ta.fisher(high=company['high'],low= company['low'],length=5)
  company['fwma'] = ta.fwma(company['close'],length=5)
  company['hilo'],company['hilol'],company['hilos']=ta.hilo(close=company['close'],high=company['high'],low= company['low'],high_length=5,low_length=5)
  company['hma'] = ta.hma(company['close'],length=5)
  company['hl2'] = ta.hl2(company['high'],company['low'])
  company['hlc3'] = ta.hlc3(company['high'],company['low'],company['close'])
  # company['hwm'],company['hwu'],company['hwl'] = ta.hwc(company['close'])
  company['hwma '] = ta.dema(company['close'])
  company['increasing'] = ta.increasing(company['close'],length=5)
  company['inertia'] = ta.inertia(high=company['high'],low= company['low'],close=company['close'],open = company['open'],rvi_length=5,length=5)
  company['jma'] = ta.jma(company['close'],length=5)
  company['kama'] = ta.kama(company['close'],length=5)
  company['kcl'],company['kcb'],company['kcu']= ta.kc(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['kdj1'],company['kdj2'],company['kdj3'] = ta.kdj(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['kurtosis'] = ta.kurtosis(company['close'],length=5)
  company['kvo'],company['kvo_s'] = ta.kvo(high=company['high'],low= company['low'],close=company['close'],volume = company['volume_traded '])
  company['linreg'] = ta.linreg(company['close'],length=5)
  company['log_return'] = ta.log_return(company['close'],length=5)
  company['macd'],company['macd_hist'],company['macd_sig'] = ta.macd(company['close'])
  company['mad'] = ta.mad(company['close'],length=5)
  company['massi'] = ta.massi(high=company['high'],low= company['low'],fast=1,slow=5)
  company['median'] = ta.median(company['close'],length=5)
  company['mfi'] = ta.mfi(high=company['high'],low= company['low'],close=company['close'],volume=company['volume_traded '],length=5)
  company['midpoint'] = ta.midpoint(company['close'],length=5)
  company['midprice'] = ta.midprice(high=company['high'],low= company['low'],length=5)
  company['mom'] = ta.mom(company['close'],length=5)
  company['natr'] = ta.natr(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['nvi'] = ta.nvi(company['close'],company['volume_traded '],length=5)
  company['obv'] = ta.obv(company['close'],company['volume_traded '])
  company['pdist'] = ta.pdist(open_=company['open'],high=company['high'],low= company['low'],close=company['close'])
  company['percent_return'] = ta.percent_return(company['close'],length=5)
  company['pgo'] = ta.pgo(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['psl'] = ta.psl(company['close'],company['open'],length=5)
  # company['pvi'] = ta.pvi(company['close'],company['volume_traded '],length=5)
  company['pvol'] = ta.pvol(company['close'],company['volume_traded '])
  company['pvt'] = ta.pvt(company['close'],company['volume_traded '])
  company['pwma'] = ta.pwma(company['close'],length=5)
  company['qqe'],company['rsi_ma'] ,company['qqel'] ,company['qqes']  = ta.qqe(company['close'],length=5)
  company['qstick'] = ta.qstick(company['open'],company['close'],length=5)
  company['quantile'] = ta.quantile(company['close'],length=5)
  company['rma'] = ta.rma(company['close'],length=5)
  company['roc'] = ta.roc(company['close'],length=5)
  company['rsi'] = ta.rsi(company['close'],length=5)
  company['rsx'] = ta.rsx(company['close'],length=5)
  # company['rvgi1'],company['rvgi2'] = ta.rvgi(open_=company['open'],high=company['high'],low= company['low'],close=company['close'],length=5)
  #company['rvil'],company['rvib'],company['rviu'],company['rvi1'],company['rvi2'],company['rvi3'],company['rvi4'],company['rvi5'],company['rvi6'],company['rvi7'],company['rvi8'],company['rvi9'],company['rvi10'],company['rvi11'] = ta.rvi(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['sinwma'] = ta.sinwma(company['close'],length=5)
  company['skew'] = ta.skew(company['close'],length=5)
  company['slope'] = ta.slope(company['close'],length=5)
  company['sma'] = ta.sma(company['close'],length=5)
  company['smi'],company['smi_s'],company['smi_osc'] = ta.smi(company['close'])
  company['ssf'] = ta.ssf(company['close'],length=5)
  company['stc'],company['stc_macd'],company['stc_stoch'] = ta.stc(company['close'],tclength=5)
  company['stdev'] = ta.stdev(company['close'],length=5)
  # company['stoch_k'],company['stoch_d'] = ta.momentum.stoch(high=company['high'],low= company['low'],close=company['close'])
  company['stochrsi_k'],company['stochrsi_d']= ta.stochrsi(company['close'],length=5)
  company['swma'] = ta.swma(company['close'],length=5)
  company['t3'] = ta.t3(company['close'],length=5)
  company['tema'] = ta.tema(company['close'],length=5)
  company['trima'] = ta.trima(company['close'],length=5)
  company['trix1'],company['trix2']  = ta.trix(company['close'],length=5)
  company['true_range'] = ta.true_range(high=company['high'],low= company['low'],close=company['close'])
  company['tema'] = ta.tema(company['close'],length=5)
  company['ui'] = ta.ui(high=company['high'],close =company['close'],length=5)
  company['uo'] = ta.uo(company['high'],company['low'],company['close'],fast=1,medium=3,slow=5)
  company['variance'] = ta.variance(company['close'],length=5)
  company['vhf'] = ta.vhf(company['close'],length=5)
  company['vidya'] = ta.vidya(company['close'],length=5)
  company['vortex1'],company['vortex2'] = ta.vortex(company['high'],company['low'],company['close'],length=5)
  # company['vp1'],company['vp2'],company['vp3'],company['vp4'],company['vp5'],company['vp6'] = ta.vp(company['close'],company['volume_traded '])
  company['vwap'] = ta.vwap(high=company['high'],low= company['low'],close=company['close'],volume=company['volume_traded '])
  company['vmwa'] = ta.vwma(volume=company['volume_traded '],close=company['close'],length=5)
  company['wcp'] = ta.wcp(high=company['high'],low= company['low'],close=company['close'])
  company['willr'] = ta.willr(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['wma'] = ta.wma(company['close'],length=5)
  company['zlma'] = ta.zlma(company['close'],length=5)
  company['zscore'] = ta.zscore(company['close'],length=5)
  #company['zlema'] = zlema(company['close'],length=5)
  company = company.fillna(0)
  #selected features based on corrlation sheet
  selected_ti = ['open','high','low','close','alma','decay','dema','ema','fwma','hma','hl2','hlc3','hwma ','jma','kama','linreg','median','midpoint','midprice','pwma','quantile','rma','sinwma','sma','ssf','swma','t3','tema','trima','vwap','vmwa','wcp','wma','zlma']
  new_company_df =  company[selected_ti]
  return new_company_df, selected_ti

### Company data prepration functions

In [20]:
# convert history into inputs and outputs
def to_supervised(X, y, n_input, n_output):
    Xs, ys = [], []
    for i in range(len(X) - n_input - n_output + 1):
        Xs.append(X.iloc[i:i+n_input].values)
        ys.append(y.iloc[i+n_input:i+n_input+n_output].values)
    return np.array(Xs), np.array(ys)

def get_last_val_set_features(data):
  # flatten data
  X = list()
  X.append(data.iloc[len(data)-5:, :])
  return X


In [21]:
#Test starting from 1-1-2017 based on 'Present Effictive short-term forecasts of Saudi stock price trends using technical indicators and large-scale multivariate time series'
# split a multivariate dataset into train/test sets
from sklearn.preprocessing import StandardScaler
from pandas import DataFrame

def prepare_company_for_training_and_testing(company, selected_ti, target_col,
                                              n_input=5, n_output=5):

    # ===============================
    # 1. Split data temporally
    # ===============================
    train = company.loc[:'2015-12-31']
    val   = company.loc['2016-01-01':'2016-12-31']
    test  = company.loc['2017-01-01':]

    print(f'Train shape: {train.shape}, Val shape: {val.shape}, Test shape: {test.shape}')

    # ===============================
    # 2. Feature scaler (X)
    # ===============================
    feature_scaler = StandardScaler()
    feature_scaler.fit(train[selected_ti])

    train_x_scaled = DataFrame(
        feature_scaler.transform(train[selected_ti]),
        columns=selected_ti,
        index=train.index
    )

    val_x_scaled = DataFrame(
        feature_scaler.transform(val[selected_ti]),
        columns=selected_ti,
        index=val.index
    )

    test_x_scaled = DataFrame(
        feature_scaler.transform(test[selected_ti]),
        columns=selected_ti,
        index=test.index
    )

    # ===============================
    # 3. Target scaler (y)
    # ===============================
    target_scaler = StandardScaler()
    target_scaler.fit(train[[target_col]])

    train_y_scaled = DataFrame(
        target_scaler.transform(train[[target_col]]),
        columns=[target_col],
        index=train.index
    )

    val_y_scaled = DataFrame(
        target_scaler.transform(val[[target_col]]),
        columns=[target_col],
        index=val.index
    )

    test_y_scaled = DataFrame(
        target_scaler.transform(test[[target_col]]),
        columns=[target_col],
        index=test.index
    )

    print("*******************************************")
    print("Train data after multistep approach")

    # ===============================
    # 4. Supervised transformation
    # ===============================
    train_x, train_y = to_supervised(train_x_scaled, train_y_scaled,
                                     n_input, n_output)

    print("Validation multistep approach")
    val_x, val_y = to_supervised(val_x_scaled, val_y_scaled,
                                 n_input, n_output)

    print("Test multistep approach")
    test_x, test_y = to_supervised(test_x_scaled, test_y_scaled,
                                   n_input, n_output)

    return (
        train_x, train_y,
        val_x, val_y,
        test_x, test_y,
        test,
        feature_scaler,
        target_scaler
    )


### Company models creating and evaluation functions

In [8]:
def true_y_to_evaluation_form(y_true, n_input, n_out):
  y = list()
  for i in range(len(y_true) - n_input - n_out + 1):
      y.append(y_true.iloc[i+n_input:i+n_input+n_out].values)
  return np.array(y)

# make a forecast
def forecast(model, history):
	yhat = model.predict(history, verbose=0)
	# we only want the vector forecast
	return yhat

# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts(actual, predicted,scaler,target_scaler):
  print(f"{len(actual)}, {len(predicted)}")
  predicted_not_normlized = target_scaler.inverse_transform(predicted)
  predicted_not_normlized_rounded = [np.round(num,2) for num in predicted_not_normlized]
  mse_score= mse(actual,predicted_not_normlized_rounded)
  rmse_score = sqrt(mse_score)
  mae_score = mae(actual,predicted_not_normlized_rounded)
  final_r2_score = r2_score(actual,predicted_not_normlized_rounded,multioutput='variance_weighted')
  evaluator = RegressionMetric(actual, predicted_not_normlized_rounded)
  mre_score = np.mean(evaluator.mean_relative_error())
  return mre_score, rmse_score, mae_score, final_r2_score, predicted_not_normlized_rounded

# summarize scores
def summarize_scores(name, mre_score, rmse_score,mae_score,final_r2_score):
  #s_scores = ', '.join(['%.1f' % s for s in scores])
  #print('%s: [%.3f] %s' % (name, score, s_scores))
  print(f'*************{name}**********')
  print("MAE: ",mae_score)
  print("MRE: ",mre_score)
  print("RMSE: ",rmse_score)
  print("R2: ",final_r2_score)

# evaluate a single model
def evaluate_model(test_x,test_y,model,scaler,target_scaler):
	# walk-forward validation over each week
  predictions = list()
  predictions = forecast(model, test_x)
  # evaluate predictions days for each week
  predictions = array(predictions)
  mse_score, rmse_score,mae_score,final_r2_score, y_hat_not_normlized = evaluate_forecasts(test_y, predictions,scaler,target_scaler)
  return mse_score, rmse_score,mae_score,final_r2_score,y_hat_not_normlized

In [33]:
def get_brnn_models_info(train_x, train_y, val_x, val_y,
                         test_x, test_y_for_evaluation,
                         company_name, evaluation_df,
                         predictions_df, scaler, target_scaler):

    verbose, epochs = 0, 32
    batch_size = math.floor(len(train_x) * 0.05)

    n_timesteps, n_features = train_x.shape[1], train_x.shape[2]

    train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))

    for i in [2,4,8,16,32,64,128,256,512,1024,
              3,9,27,81,243,729,
              100,200,300,400,500,600,700,800,900,1000]:

        model = Sequential()
        model.add(
            Bidirectional(
                SimpleRNN(i, activation='relu',
                          input_shape=(n_timesteps, n_features))
            )
        )
        model.add(Dense(5))
        model.compile(loss='mse', optimizer='adam')

        model.fit(
            train_x, train_y,
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
            validation_data=(val_x, val_y)
        )

        mre, rmse, mae_s, r2, preds = evaluate_model(
            test_x, test_y_for_evaluation,
            model, scaler, target_scaler
        )

        evaluation_df.loc[i, ['MAE_b','MRE_b','RMSE_b','R2_b']] = \
            mae_s, mre, rmse, r2

        # predictions_df[f"BRNN_{i}"] = preds.flatten()

        # pickle.dump(
        #     model,
        #     open(f'/content/drive/MyDrive/KFUPM/Graduation/models/{company_name}_BRNN_{i}', 'wb')
        # )

        summarize_scores('Proposed BRNN', mre, rmse, mae_s, r2)

    return evaluation_df, predictions_df


def get_gru_models_info(train_x, train_y, val_x, val_y,
                        test_x, test_y_for_evaluation,
                        company_name, evaluation_df,
                        predictions_df, scaler, target_scaler):

    verbose, epochs = 0, 32
    batch_size = math.floor(len(train_x) * 0.05)

    n_timesteps, n_features = train_x.shape[1], train_x.shape[2]
    train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))

    for i in [2,4,8,16,32,64,128,256,512,1024,
              3,9,27,81,243,729,
              100,200,300,400,500,600,700,800,900,1000]:

        model = Sequential()
        model.add(GRU(i, activation='relu',
                      input_shape=(n_timesteps, n_features)))
        model.add(Dense(5))
        model.compile(loss='mse', optimizer='adam')

        model.fit(
            train_x, train_y,
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
            validation_data=(val_x, val_y)
        )

        mre, rmse, mae_s, r2, preds = evaluate_model(
            test_x, test_y_for_evaluation,
            model, scaler, target_scaler
        )

        evaluation_df.loc[i, ['MAE_g','MRE_g','RMSE_g','R2_g']] = \
            mae_s, mre, rmse, r2

        # predictions_df[f"GRU_{i}"] = preds.flatten()

        # pickle.dump(
        #     model,
        #     open(f'/content/drive/MyDrive/KFUPM/Graduation/models/{company_name}_GRU_{i}', 'wb')
        # )

        summarize_scores('Proposed GRU', mre, rmse, mae_s, r2)

    return evaluation_df, predictions_df

def get_lstm_models_info(train_x, train_y, val_x, val_y,
                         test_x, test_y_for_evaluation,
                         company_name, evaluation_df,
                         predictions_df, scaler, target_scaler):

    verbose, epochs = 0, 32
    batch_size = math.floor(len(train_x) * 0.05)

    n_timesteps, n_features = train_x.shape[1], train_x.shape[2]
    train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))

    for i in [2,4,8,16,32,64,128,256,512,1024,
              3,9,27,81,243,729,
              100,200,300,400,500,600,700,800,900,1000]:

        model = Sequential()
        model.add(LSTM(i, activation='relu',
                       input_shape=(n_timesteps, n_features)))
        model.add(Dense(5))
        model.compile(loss='mse', optimizer='adam')

        model.fit(
            train_x, train_y,
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
            validation_data=(val_x, val_y)
        )

        mre, rmse, mae_s, r2, preds = evaluate_model(
            test_x, test_y_for_evaluation,
            model, scaler, target_scaler
        )

        evaluation_df.loc[i, ['MAE_l','MRE_l','RMSE_l','R2_l']] = \
            mae_s, mre, rmse, r2

        # predictions_df[f"LSTM_{i}"] = preds.flatten()

        # pickle.dump(
        #     model,
        #     open(f'/content/drive/MyDrive/KFUPM/Graduation/models/{company_name}_LSTM_{i}', 'wb')
        # )

        summarize_scores('Proposed LSTM', mre, rmse, mae_s, r2)

    return evaluation_df, predictions_df


In [27]:
def train_evalute_models_for_company(train_x, train_y, val_x, val_y,
                                     test_x, test,
                                     company_name, scaler, target_scaler):

    eval_cols = [
        'MAE_b','MRE_b','RMSE_b','R2_b',
        'MAE_g','MRE_g','RMSE_g','R2_g',
        'MAE_l','MRE_l','RMSE_l','R2_l'
    ]

    unit_indices = [
        2,4,8,16,32,64,128,256,512,1024,
        3,9,27,81,243,729,
        100,200,300,400,500,600,700,800,900,1000
    ]

    test_y_for_evaluation = true_y_to_evaluation_form(
        test['close'], 5, 5
    )

    evaluation_df = pd.DataFrame(columns=eval_cols, index=unit_indices)

    predictions_df = DataFrame(
        data=test_y_for_evaluation.flatten(),
        columns=['Actual']
    )

    evaluation_df, predictions_df = get_lstm_models_info(
        train_x,train_y,val_x,val_y,
        test_x,test_y_for_evaluation,
        company_name,evaluation_df,predictions_df,
        scaler,target_scaler
    )

    evaluation_df, predictions_df = get_brnn_models_info(
        train_x,train_y,val_x,val_y,
        test_x,test_y_for_evaluation,
        company_name,evaluation_df,predictions_df,
        scaler,target_scaler
    )

    evaluation_df, predictions_df = get_gru_models_info(
        train_x,train_y,val_x,val_y,
        test_x,test_y_for_evaluation,
        company_name,evaluation_df,predictions_df,
        scaler,target_scaler
    )

    return evaluation_df, predictions_df


## Saudi Excution

In [11]:
dataset = pd.read_csv('/content/drive/MyDrive/KFUPM/Graduation/data/Saudi/Tadawul_stcks.csv',index_col='date',parse_dates=True)
dataset.sort_values(by='date', inplace = True)

In [12]:
selected_companies_dic = {"ADC":["ADC","2001-12-31"], "ALRAJHI":["ALRAJHI","2001-12-31"], "GACO":["GACO","2002-01-01"], "EMAAR EC":["EMAAR EC","2006-10-06"], "FITAIHI GROUP":["FITAIHI GROUP","2002-01-26"], "SARCO":["SARCO","2002-01-23"], "SAUDI ELECTRICITY":["SAUDI ELECTRICITY","2002-06-08"], "SPIMACO":["SPIMACO","2001-12-31"], "STC":["STC","2003-01-25"], "TASNEE":["TASNEE","2001-12-31"] }

In [ ]:
evaluation_df = pd.DataFrame()
for key, val in selected_companies_dic.items():
  co, co_selected_ti = get_company_df(dataset,selected_companies_dic[key][0],selected_companies_dic[key][1])
  train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test,scaler,target_scaler = prepare_company_for_training_and_testing(co, co_selected_ti,'close')
  train_evalute_models_for_company(train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test,selected_companies_dic[key][0],scaler,target_scaler)
#evaluation_df.to_csv("/content/drive/MyDrive/KFUPM/Graduation/final_experiments/models/NormlizationCorrectionResults/result.csv",index=True)

/tmp/ipython-input-3012923053.py:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return pd.DataFrame(values).fillna(method='ffill').values
/usr/local/lib/python3.12/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.12/dist-packages/pandas_ta/overlap/linreg.py:120: RuntimeWarning: invalid value encountered in scalar power
  rd = (divisor * (length * y2_sum - y_sum * y_sum)) ** 0.5
/tmp/ipython-input-1125527301.py:71: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  company['midprice'] = ta.midprice(high=company['high'],low= company['low'],length=5)


Train shape: (5114, 34), Val shape: (366, 34), Test shape: (1160, 34)
*******************************************
Train data after multistep approach
Validation multistep approach
Test multistep approach
1151, 1151
*************Proposed LSTM**********
MAE:  0.3507976830005646
MRE:  0.02973
RMSE:  0.4872084467154881
R2:  0.8882942199707031
1151, 1151
*************Proposed LSTM**********
MAE:  0.29734134674072266
MRE:  0.024574000000000002
RMSE:  0.4564969779120331
R2:  0.9019333124160767
1151, 1151
*************Proposed LSTM**********
MAE:  0.23444819450378418
MRE:  0.019894000000000002
RMSE:  0.33498974111847807
R2:  0.9471908807754517
1151, 1151
*************Proposed LSTM**********
MAE:  0.2216334044933319
MRE:  0.01892
RMSE:  0.3183420796757293
R2:  0.952309250831604
1151, 1151
*************Proposed LSTM**********
MAE:  0.18761244416236877
MRE:  0.016054
RMSE:  0.27612303338358585
R2:  0.9641199707984924
1151, 1151
*************Proposed LSTM**********
MAE:  0.17994442582130432
MRE:  

/tmp/ipython-input-3012923053.py:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return pd.DataFrame(values).fillna(method='ffill').values
/usr/local/lib/python3.12/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.12/dist-packages/pandas_ta/overlap/linreg.py:120: RuntimeWarning: invalid value encountered in scalar power
  rd = (divisor * (length * y2_sum - y_sum * y_sum)) ** 0.5
/tmp/ipython-input-1125527301.py:71: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  company['midprice'] = ta.midprice(high=company['high'],low= company['low'],length=5)


Train shape: (5114, 34), Val shape: (366, 34), Test shape: (1160, 34)
*******************************************
Train data after multistep approach
Validation multistep approach
Test multistep approach
1151, 1151
*************Proposed LSTM**********
MAE:  2.3733365535736084
MRE:  0.048912
RMSE:  2.937289940648449
R2:  0.9220336675643921
1151, 1151
*************Proposed LSTM**********
MAE:  1.639120101928711
MRE:  0.031866
RMSE:  2.025802702814062
R2:  0.9629141092300415
1151, 1151
*************Proposed LSTM**********
MAE:  0.9133817553520203
MRE:  0.016848
RMSE:  1.2616743428354975
R2:  0.9856149554252625
1151, 1151
*************Proposed LSTM**********
MAE:  1.0158106088638306
MRE:  0.01802
RMSE:  1.388520909941462
R2:  0.9825771450996399
1151, 1151
*************Proposed LSTM**********
MAE:  0.8392635583877563
MRE:  0.015113999999999999
RMSE:  1.1918161156225575
R2:  0.9871638417243958
1151, 1151
*************Proposed LSTM**********
MAE:  0.9161025881767273
MRE:  0.016458
RMSE:  1.29

/tmp/ipython-input-3012923053.py:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return pd.DataFrame(values).fillna(method='ffill').values
/usr/local/lib/python3.12/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.12/dist-packages/pandas_ta/overlap/linreg.py:120: RuntimeWarning: invalid value encountered in scalar power
  rd = (divisor * (length * y2_sum - y_sum * y_sum)) ** 0.5
/usr/local/lib/python3.12/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.12/dist-packages/pandas_ta/performance/log_return.py:48: RuntimeWarning: divide by zero encountered in divide
  r = np_close / roll(np_close, length)
/usr/local/lib/python3.12/dist-packages/pandas_ta/perfo

Train shape: (5113, 34), Val shape: (366, 34), Test shape: (1160, 34)
*******************************************
Train data after multistep approach
Validation multistep approach
Test multistep approach
1151, 1151
*************Proposed LSTM**********
MAE:  1.870282530784607
MRE:  0.158774
RMSE:  3.1949999966569163
R2:  -4.819623947143555
1151, 1151
*************Proposed LSTM**********
MAE:  1.816451072692871
MRE:  0.156544
RMSE:  2.7429988345520235
R2:  -3.2894768714904785
1151, 1151
*************Proposed LSTM**********
MAE:  0.29008156061172485
MRE:  0.024259999999999997
RMSE:  0.379681923907743
R2:  0.9178148508071899
1151, 1151
*************Proposed LSTM**********
MAE:  0.27942126989364624
MRE:  0.023256000000000002
RMSE:  0.36420260258985376
R2:  0.9243794679641724
1151, 1151
*************Proposed LSTM**********
MAE:  0.23517103493213654
MRE:  0.019686000000000002
RMSE:  0.32002984345496427
R2:  0.94161057472229
1151, 1151
*************Proposed LSTM**********
MAE:  0.3717097640037